In [ ]:
import cv2
import tensorflow as tf

imported1 = tf.saved_model.load("../input/google-2020-baseline/∩╗┐basline")
f1 = imported1.signatures["serving_default"]

input_image = tf.constant(cv2.imread('../input/landmark-retrieval-2020/train/0/0/0/0000059611c7d079.jpg'), dtype=tf.uint8)

In [ ]:
i1 = f1(input_image)['global_descriptor']#.numpy()
i2 = f1(tf.image.flip_left_right(input_image))['global_descriptor']#.numpy()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.hist(tf.abs((i1-i2)/(tf.abs(i1)+0.00001+tf.abs(i2))))

In [ ]:
d = tf.math.abs((i1-i2)/(tf.math.abs(i1)+tf.math.abs(i2)+0.00001))
d = tf.cast(d<0.9, dtype=tf.float32)
o = i1+0.5*i2
o = o*d

In [ ]:
o

In [ ]:
i1+0.5*i2

In [ ]:
model = tf.saved_model.load("../input/google-2020-baseline/∩╗┐basline")

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model1 = model
        self.f1 = self.model1.signatures["serving_default"]
        
    @tf.function(input_signature=[
      tf.TensorSpec(shape=[None, None, 3], dtype=tf.uint8, name='input_image')
    ])
    def call(self, input_image):
        output_tensors = {}
        i1 = self.f1(input_image)['global_descriptor']
        i2 = self.f1(tf.image.flip_left_right(input_image))['global_descriptor']
        d = tf.math.abs((i1-i2)/(tf.math.abs(i1)+tf.math.abs(i2)+0.00001))
        d = tf.cast(d<0.9, dtype=tf.float32)
        o = i1+0.5*i2
        o = o*d
        output_tensors['global_descriptor'] = tf.identity( o , name='global_descriptor')
        return output_tensors

In [ ]:
m = MyModel() #creating our model instance

served_function = m.call
tf.saved_model.save(
      m, export_dir="./my_model", signatures={'serving_default': served_function})

In [ ]:
!ls ./my_model/assets

In [ ]:
!saved_model_cli show --dir "../input/google-2020-baseline/∩╗┐basline" --all

In [ ]:
!saved_model_cli show --dir ./my_model/ --all

In [ ]:
imported3 = tf.saved_model.load("./my_model")
f3 = imported3.signatures["serving_default"]

f3(input_image)

In [ ]:
!ls ./my_model/variables

In [ ]:
from zipfile import ZipFile

with ZipFile('submission.zip','w') as zip:           
    zip.write('./my_model/saved_model.pb', arcname='saved_model.pb')
    zip.write('./my_model/variables/variables.data-00000-of-00001', arcname='variables/variables.data-00000-of-00001') 
    zip.write('./my_model/variables/variables.index', arcname='variables/variables.index') 

In [ ]:
!ls "./my_model/assets"